# Post-analysis of Spindles, SWR and Down-states.

Restart from SWR and spindles verified files. They are most likely the .csv files.

## Load events files and packages

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
import seaborn as sns

sns.set(style="whitegrid", palette="muted")
%matplotlib widget

SpindlesS1 = pd.read_csv('./EventsFiles/Spindleproperties_S1_1.csv')
SpindlesPFC = pd.read_csv('./EventsFiles/Spindleproperties_PFC_1.csv')
SWRCA1 = pd.read_csv('./EventsFiles/SWRproperties_1.csv')
LFPwakeremoved = np.load('LFPwakeremoved_2.npy', mmap_mode= 'r')
All = np.load('RawDataChannelExtractedDS_2.npy', mmap_mode= 'r')
PFC = All[:, 1]
PFCwakeremoved = LFPwakeremoved[:,0]


In [ ]:
SpindlesS1

In [ ]:
SpindlesPFC

In [ ]:
SWRCA1
SWRCA1.describe()
r = np.corrcoef(SWRCA1['Duration'], SWRCA1['Avg int'])
r[0, 1] # -0.1757 No correlation!
r = np.corrcoef(SWRCA1['Avg freq'], SWRCA1['Avg int'])
r[0, 1] # 0.2528 No correlation!
r = np.corrcoef(SWRCA1['Avg freq'], SWRCA1['Duration'])
r[0, 1] # -0.2478 No correlation!

In [ ]:
fig, ax = plt.subplots()
ax.plot(SWRCA1['Duration'], SWRCA1['Avg int'], marker='o', linewidth=0)
ax.set_title('correlation Duration - Intensity')
ax.set_xlabel('Duration (ms)')
ax.set_ylabel('Intensity (AU)')